In [ ]:
from pso.pso import PSO
import matplotlib.pyplot as plt
import numpy as np

dim = 4
p_num = 20
x_min, x_max = -30, 30
v_min, v_max = -60, 60

PSO_Algorithm = PSO(C1 = 2, 
                    C2 = 2, 
                    Omega = 1,
                    dim = dim,
                    p_num = p_num, 
                    x_min = x_min, 
                    x_max = x_max, 
                    v_min = v_min, 
                    v_max = v_max
                )
for par in PSO_Algorithm.particles:
    print(f'particle init pos is {par.get_pos()}')
max_iter = 10000

def cal_fitness(x):
    return np.sum(x**2)
    return sum(100.0 * (x[0][1:] - x[0][:-1] ** 2.0) ** 2.0 + (1 - x[0][:-1]) ** 2.0)

for i in range(max_iter):
    print(f'iter is {i+1}, fitness is {cal_fitness(PSO_Algorithm.g_best)}')
    PSO_Algorithm.update(cal_fitness)

In [ ]:
PSO_Algorithm.g_best

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation   #导入负责绘制动画的接口
#其中需要输入一个更新数据的函数来为fig提供新的绘图信息

fig, ax = plt.subplots()          #生成轴和fig,  可迭代的对象
x, y= [], []    #用于接受后更新的数据
line, = plt.plot([], [], '.-')   #绘制线对象，plot返回值类型，要加逗号

#------说明--------#
#核心函数包含两个：
#一个是用于初始化画布的函数init()
#另一个是用于更新数据做动态显示的update()


def init():
	#初始化函数用于绘制一块干净的画布，为后续绘图做准备
    ax.set_xlim(-5, 15*np.pi)    #初始函数，设置绘图范围
    ax.set_ylim(-3, 3)
    return line

def update(step):           #通过帧数来不断更新新的数值
    x.append(step)
    y.append(np.cos(step/3)+np.sin(step**2))    #计算y
    line.set_data(x, y)
    return line

#fig 是绘图的画布
#update 为更新绘图的函数，step数值是从frames 传入
#frames 数值是用于动画每一帧的数据
ani = FuncAnimation(fig, update, frames=np.linspace(0, 13*np.pi, 128),
                    init_func=init,interval=20)

plt.show()


In [2]:
class PPO(object):
    def __init__(self,state_dim, act_dim, hidden_dims,layer_nums,train_params) -> None:
        self.ppo_params = {
            # 网络层参数
            'state_dim' : state_dim,
            'act_dim' : act_dim,
            'layer_nums' : layer_nums,

            # 训练参数
            'lr_a': train_params['lr_a'],
            'lr_c': train_params['lr_c'],
            'batch_size' : train_params['batch_size'],
            'on_policy' : False,
            'use_buffer' : False,
            'use_tanh' : train_params['use_tanh']
        }

        if not isinstance(hidden_dims,list) :
            raise RuntimeError(f"hidden_dims type must be list, now receive {type(hidden_dims)}. ")
        if len(hidden_dims) != layer_nums - 1:
            raise RuntimeError(f"hidden_dims'len expect {layer_nums-1}, but now receive {len(hidden_dims)}. ")
        

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
for index in BatchSampler(range(1024), 64, False):
    print(index)

In [55]:
import gym
# from share_func import make_env
# train_envs = [ make_env(env_name = "CartPole-v1", seed = 1,idx = i,capture_video = False, run_name = f'_video{i}') for i in range(1000) ]
def make_env(gym_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk
train_envs = [ make_env(gym_id = "CartPole-v1", seed = 1,idx = i,capture_video = False, run_name = f'_video{i}') for i in range(10) ]
envs = gym.vector.SyncVectorEnv(train_envs)

In [ ]:
gym.make("CartPole-v1")._max_episode_steps

In [ ]:
envs.single_action_space.n

In [ ]:
# env reset, state shape is (env_num, state_dim)
from ppo.ppo import PPO
import torch
import numpy as np
ppo_params = {
        # ppo algorithm params
        'clip_param' : 0.2,

        # 训练参数
        'lr_a': 1e-3,
        'lr_c': 1e-3,
        'gamma': 0.5,
        'lamda': 0.5,
        'batch_size' : 1024,
        'mini_batch_size': 64,

        # trick params

        'off_policy' : False, # use off-policy or on-policy
        'use_buffer' : True, # use buffer to store or not
        "use_ppo_clip":False , # use ppo clip param annealing
        "use_adv_norm" : True, # use advantage normalization
        "use_state_norm" : False, # use state normalization
        "use_reward_norm" : False, # use reward normalization
        'use_tanh' : False, # use tanh activate func or ReLU func
        'use_adv_norm' : False, # use advantage normalization
        'use_grad_clip' : True, # use grad clip in model params.
        'grad_clip_params': 0.5,
        'use_lr_decay': True,
        'entropy_coef': 0.1,
        'device': torch.device("cuda"),        
}
agent = PPO(state_dim = 4,act_dim = 2, hidden_dims = [64,64], layer_nums = 3, train_params = ppo_params)
state, _ = envs.reset()
done = np.full((10,1), False)
while np.sum(done) == 0:
    print(state, np.sum(done),sep="\t\t")
    action, a_logprob = agent.select_action(state)
    state_, reward, done, truncation, _ = envs.step(action)
    # replay_buffer.add(state = state, action = action, reward = reward, next_state = state_, a_logprob = a_logprob, done = done)
    if np.sum(done) == envs.action_space.nvec.shape[0] :
        print(f'done is {done}')
        print(f'state shape is {state_.shape}, \nstate is {state_}')
    state = state_
    

In [ ]:
import cv2
 
cap = cv2.VideoCapture('XXX.avi')  #返回一个capture对象
cap.set(cv2.CAP_PROP_POS_FRAMES,50)  #设置要获取的帧号
a,b=cap.read()  #read方法返回一个布尔值和一个视频帧。若帧读取成功，则返回True
cv2.imshow('b', b)
cv2.waitKey(1000)

In [1]:
import torch
import numpy as np
t1 = torch.Tensor(np.ones((2,5)))
t2 = torch.Tensor(np.zeros((2,1)))

In [5]:
a = torch.Tensor([[1,2,3,4,5,6],[0,1,2,3,4,5]])
b = torch.tensor([[5,4,3,2,1,0],[0,1,2,3,4,5]])

In [6]:
c = a.gather(1,b)

In [7]:
c

tensor([[6., 5., 4., 3., 2., 1.],
        [0., 1., 2., 3., 4., 5.]])

In [9]:
torch.max(a, axis=0)

torch.return_types.max(
values=tensor([1., 2., 3., 4., 5., 6.]),
indices=tensor([0, 0, 0, 0, 0, 0]))